In [38]:
# Part 1.1
import requests
from bs4 import BeautifulSoup

In [39]:
# Part 1.2
response = requests.get('https://kurser.ku.dk/course/NNDB19000U')
contents = response.text
soup = BeautifulSoup(contents, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html lang="da">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="Københavns Universitet er med cirka 40.000 studerende og 9.000 medarbejdere en af Nordens største forsknings- og uddannelsesinstitutioner." name="description"/>
  <meta content="Københavns Universitet" name="author"/>
  <title>
   Datalogiens videnskabsteori (VtDat)
  </title>
  <link href="https://bootstrap.secure.ku.dk/img/apple-touch-icon.png" rel="apple-touch-icon"/>
  <link href="https://bootstrap.secure.ku.dk/img/favicon_fa.ico" rel="icon"/>
  <link href="https://cms.ku.dk/bootstrap/dist/css/bootstrap.min.css" rel="stylesheet"/>
  <link href="https://bootstrap.secure.ku.dk/css/multiple-select.min.css" rel="stylesheet"/>
  <link href="https://bootstrap.secure.ku.dk/css/datatables.min.css" rel="stylesheet"/>
  <link href="https:

In [40]:
title = soup.find('title')
print(title)

<title>
Datalogiens videnskabsteori (VtDat)</title>


In [41]:
course_info = soup.find('div', class_="panel-body")
# every "dt" contains a key
# every dd contains a value
keys = course_info.find_all('dt')
values = course_info.find_all('dd')
# make table
course_table = {}
course_table['title'] = title.text
for i in range(len(keys)):
    course_table[keys[i].text] = values[i].text

In [42]:
# id is course-exams1
exam_info = soup.find("div", id="course-exams1")
# Exam type is either "exam type" or "prøveform" case insensitive
exam_type = exam_info.find("dt", string=lambda x: x.lower() == "exam type" or x.lower() == "prøveform")
# Find the dd immediately after the dt and return the divs
exam_type = exam_type.find_next_sibling("dd")
course_table['exam type'] = exam_type

In [43]:
# print each row of table with newline
for key, value in course_table.items():
    print(key, ":", value, "\n")

title : 
Datalogiens videnskabsteori (VtDat) 

Sprog : Dansk 

Kursuskode : NNDB19000U 

Point : 7,5 ECTS 

Niveau : Bachelor 

Varighed : 1 blok 

Placering : Blok 4 

Skemagruppe : B 

Kursuskapacitet : Ingen begrænsning

Der kan være færre pladser i eftertilmeldingsperioden 

exam type : Skriftlig prøve, 60 min med opsyn.Skriftlig aflevering, 48 timer 

